In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/spaceship-titanic/sample_submission.csv
/kaggle/input/spaceship-titanic/train.csv
/kaggle/input/spaceship-titanic/test.csv


In [2]:
# Train data preparation
# 1. Split cabin info as it has deck and side , num value is not importance as it is loosely correlated to transported
# 2. Combine all expenses into Bill column
# 3. Replace True to 1 and False to 0
# 4. Categorize Age and bill using histogram and normal distribution.
train_data = pd.read_csv("/kaggle/input/spaceship-titanic/train.csv")
cabin_splits = train_data["Cabin"].str.split("/",n=2,expand=True)
train_data[["CabinDeck","CabinNum","CabinSide"]] = cabin_splits
train_data.drop(columns=["Cabin","CabinNum","Name"],inplace=True)
for col in ["RoomService","FoodCourt","ShoppingMall","Spa","VRDeck","Age"] :
    train_data[col].fillna(train_data[col].mean(),inplace=True)
train_data["Bill"] = train_data["RoomService"]+train_data["FoodCourt"]+train_data["ShoppingMall"]+train_data["Spa"]+train_data["VRDeck"]
train_data.drop(columns=["RoomService","FoodCourt","ShoppingMall","Spa","VRDeck"],inplace=True)
for col in ["CryoSleep","VIP","HomePlanet","Destination","CabinDeck","CabinSide"] :
    train_data[col].fillna(train_data[col].mode()[0],inplace=True)
    train_data[col].replace({ True : 1 , False : 0 },inplace=True)
train_data.loc[train_data["Bill"]<=1250,"Bill"] = 0
train_data.loc[(train_data["Bill"]<=5000) & (train_data["Bill"]>1250),"Bill"] = 1
train_data.loc[train_data["Bill"]>5000,"Bill"] = 2
train_data.loc[train_data["Age"]<=20,"Age"] = 0
train_data.loc[(train_data["Age"]<=40) & (train_data["Age"]>20),"Age"] = 1
train_data.loc[train_data["Age"]>40,"Age"] = 2

In [3]:
# Test data preparation
test_data = pd.read_csv("/kaggle/input/spaceship-titanic/test.csv")
print("data size before - "+str(len(test_data)))
cabin_splits = test_data["Cabin"].str.split("/",n=2,expand=True)
test_data[["CabinDeck","CabinNum","CabinSide"]] = cabin_splits
test_data.drop(columns=["Cabin","CabinNum","Name"],inplace=True)
for col in ["RoomService","FoodCourt","ShoppingMall","Spa","VRDeck","Age"] :
    test_data[col].fillna(test_data[col].mean(),inplace=True)
test_data["Bill"] = test_data["RoomService"]+test_data["FoodCourt"]+test_data["ShoppingMall"]+test_data["Spa"]+test_data["VRDeck"]
test_data.drop(columns=["RoomService","FoodCourt","ShoppingMall","Spa","VRDeck"],inplace=True)
for col in ["CryoSleep","VIP","HomePlanet","Destination","CabinDeck","CabinSide"] :
    test_data[col].fillna(test_data[col].mode()[0],inplace=True)
    test_data[col].replace({ True : 1 , False : 0 },inplace=True)
test_data.loc[test_data["Bill"]<=1250,"Bill"] = 0
test_data.loc[(test_data["Bill"]<=5000) & (test_data["Bill"]>1250),"Bill"] = 1
test_data.loc[test_data["Bill"]>5000,"Bill"] = 2
test_data.loc[test_data["Age"]<=20,"Age"] = 0
test_data.loc[(test_data["Age"]<=40) & (test_data["Age"]>20),"Age"] = 1
test_data.loc[test_data["Age"]>40,"Age"] = 2
print("data size after - "+str(len(test_data)))

data size before - 4277
data size after - 4277


In [4]:
from sklearn.ensemble import RandomForestClassifier
y = train_data["Transported"]
X_train = pd.get_dummies(train_data.drop(columns=["Transported","PassengerId"]))
X_test = pd.get_dummies(test_data.drop(columns=["PassengerId"]))
model = RandomForestClassifier(n_estimators=5000, max_depth=30, random_state=1)
model.fit(X_train, y)
predictions = model.predict(X_test)
output = pd.DataFrame({'PassengerId': test_data.PassengerId, 'Transported': predictions})
output.to_csv('submission.csv', index=False)
print(model.score(X_train,y))

0.7592315656275164
